In [1]:
import os 
import pandas as pd 
import numpy as np 
os.chdir('/mnt/BioHome/jreyna/jreyna/projects/cmi-pb-preds/')
outdir = 'results/main/cmi_pb_datasets/processed/'
os.makedirs(outdir, exist_ok=True)

In [2]:
assays = ('cytof', 'olink', 'rnaseq', 'abtiters')
longnames = ('live_cell_percentages', 'olink_prot_exp', 'rnaseq', 'ab_titer')

In [3]:
# loading gencode data
gencode = pd.read_table('results/refs/gencode/gencode.v38lift37.annotation.protein_coding.bed', header=None)
gencode.columns = ['chr', 'start', 'end', 'strand', 'gene_id', 'gene_name']
gencode['gene_nonversioned_id'] = gencode['gene_id'].str.replace('\..*', '')

In [4]:
interm_dir = os.path.join(outdir, 'full')

## Load the 2020 data

In [5]:
twenty_data = {}

# get meta master table
for metatable in ('subject', 'specimen'): 
    fn = 'results/main/cmi_pb_datasets/raw/2020LD_{}.csv'.format(metatable)
    df = pd.read_table(fn, sep=',')
    twenty_data[metatable] = df
    
subjects = twenty_data['subject']
specimen = twenty_data['specimen']
master_meta = subjects.merge(specimen, on='subject_id')
master_meta = master_meta[['subject_id',
                           'specimen_id',
                           'infancy_vac',
                           'biological_sex',
                           'year_of_birth',
                           'date_of_boost',
                           'actual_day_relative_to_boost',
                           'planned_day_relative_to_boost',
                           'ethnicity',
                           'race',
                           'dataset',
                           'specimen_type',
                           'visit']]

# defining the meta columns (used to remove columns later one)
meta_cols = ['specimen_id', 'infancy_vac', 'biological_sex',
             'year_of_birth', 'date_of_boost', 'actual_day_relative_to_boost',
             'ethnicity', 'race', 'dataset',
             'specimen_type', 'visit']

twenty_data['master_meta'] = master_meta

# save meta
meta_fn = os.path.join(interm_dir, 'meta.2020.pivoted.tsv.gz')
twenty_data['master_meta'].to_csv(meta_fn, sep='\t')

In [6]:
for i, longname in enumerate(longnames): 
    fn = 'results/main/cmi_pb_datasets/raw/2020LD_{}.csv'.format(longname)
    df = pd.read_table(fn, sep=',')
    
    print(fn)
    
    if assays[i] == 'rnaseq':
        df.loc[:, 'ensembl_gene_id'] = df.loc[:, 'versioned_ensembl_gene_id'].str.replace('\.[0-9]+', '')
        df = df.pivot(index='specimen_id', columns='ensembl_gene_id', values='tpm')
        df = master_meta.merge(df, on='specimen_id')
        
    elif assays[i] == 'cytof':
        df = df.pivot(index='specimen_id', columns='cell_type_name', values='percent_live_cell')
        df = master_meta.merge(df, on='specimen_id')
        
    elif assays[i] == 'olink':
        # Still getting duplicate specimens? 
        df = df[df['unit'] == "Normalized Protein eXpression"]
        df = df[df['quality_control'] == 'Pass']
        df = df[df['protein_expression'] > df['lower_limit_of_quantitation']]
        df = df.pivot(index='specimen_id', columns='olink_id', values='protein_expression')
        df = master_meta.merge(df, on='specimen_id')      
        
    else:
        aglist = ['PT', 'PRN', 'FHA', 'FIM2/3']
        df = df.loc[df.antigen.isin(aglist), :]
        df['isotype_antigen'] = df['isotype'] + '-' + df['antigen']
        df = df.pivot(index='specimen_id', columns='isotype_antigen', values='ab_titer')
        df = master_meta.merge(df, on='specimen_id')  

    twenty_data[assays[i]] = df.drop(meta_cols, axis=1)
    

results/main/cmi_pb_datasets/raw/2020LD_live_cell_percentages.csv
results/main/cmi_pb_datasets/raw/2020LD_olink_prot_exp.csv
results/main/cmi_pb_datasets/raw/2020LD_rnaseq.csv
results/main/cmi_pb_datasets/raw/2020LD_ab_titer.csv


In [7]:
for assay, df in twenty_data.items():
    print(assay, df.subject_id.nunique())

subject 60
specimen 60
master_meta 60
cytof 20
olink 18
rnaseq 36
abtiters 58


In [8]:
# save a dataframe for each assay on each day
for assay, df in twenty_data.items():
    if assay in ['cytof', 'olink', 'rnaseq', 'abtiters']:
        print(assay)
        for day, day_df in df.groupby('planned_day_relative_to_boost'):
            outfn = os.path.join(interm_dir, '{}.2020.day{}.pivoted.tsv.gz'.format(assay, day))
            day_df.drop('planned_day_relative_to_boost', axis=1).to_csv(outfn, index=False, sep='\t')
            print(day, day_df.shape)

cytof
0 (20, 27)
1 (20, 27)
3 (20, 27)
7 (20, 27)
14 (20, 27)
olink
0 (18, 256)
1 (18, 256)
3 (18, 256)
7 (18, 256)
14 (18, 256)
rnaseq
0 (36, 58349)
1 (36, 58349)
3 (36, 58349)
7 (36, 58349)
14 (36, 58349)
abtiters
0 (58, 20)
1 (57, 20)
3 (57, 20)
7 (57, 20)
14 (57, 20)
30 (54, 20)
90 (51, 20)
386 (1, 20)
402 (1, 20)
428 (1, 20)


# Load the 2021 data

In [9]:
twentyone_data = {}

# get meta master table
for metatable in ('subject', 'specimen'): 
    fn = 'results/main/cmi_pb_datasets/raw/2021BD_{}.csv'.format(metatable)
    df = pd.read_table(fn, sep=',')
    twentyone_data[metatable] = df
    
subjects = twentyone_data['subject']
specimen = twentyone_data['specimen']
master_meta = subjects.merge(specimen, on='subject_id')
master_meta = master_meta[['subject_id',
                           'specimen_id',
                           'infancy_vac',
                           'biological_sex',
                           'year_of_birth',
                           'date_of_boost',
                           'actual_day_relative_to_boost',
                           'planned_day_relative_to_boost',
                           'ethnicity',
                           'race',
                           'dataset',
                           'specimen_type',
                           'visit']]
twentyone_data['master_meta'] = master_meta

In [10]:
for i, longname in enumerate(longnames): 
        
    fn = 'results/main/cmi_pb_datasets/raw/2021BD_{}.csv'.format(longname)
    df = pd.read_table(fn, sep=',')
    
    if assays[i] == 'rnaseq':
        df.loc[:, 'ensembl_gene_id'] = df.loc[:, 'versioned_ensembl_gene_id'].str.replace('\.[0-9]+', '')
        df = df.pivot(index='specimen_id', columns='ensembl_gene_id', values='tpm')
        df = master_meta.merge(df, on='specimen_id')
                
    elif assays[i] == 'cytof':
        df = df.pivot(index='specimen_id', columns='cell_type_name', values='percent_live_cell')
        df = master_meta.merge(df, on='specimen_id')
        
    elif assays[i] == 'olink':
        df = df.pivot(index='specimen_id', columns='olink_id', values='protein_expression')
        df = master_meta.merge(df, on='specimen_id')       
        
    else:
        aglist = ['PT', 'PRN', 'FHA', 'FIM2/3']
        df = df.loc[df.antigen.isin(aglist), :]
        df['isotype_antigen'] = df['isotype'] + '-' + df['antigen']
        df = df.pivot(index='specimen_id', columns='isotype_antigen', values='ab_titer')
        df = master_meta.merge(df, on='specimen_id')  
          
    twentyone_data[assays[i]] = df.drop(meta_cols, axis=1)
    

In [11]:
longname = 'live_cell_percentages'
df = 'results/main/cmi_pb_datasets/raw/2021BD_{}.csv'.format(longname)

df = pd.read_table(df, sep=',')

df = df.pivot(index='specimen_id', columns='cell_type_name', values='percent_live_cell')
df = master_meta.merge(df, on='specimen_id')

        

In [12]:
for assay, df in twentyone_data.items():
    print(assay, df.subject_id.nunique())

subject 36
specimen 36
master_meta 36
cytof 33
olink 36
rnaseq 36
abtiters 33


In [13]:
os.makedirs(interm_dir, exist_ok=True)

# save abtiters  
abtiters_fn = os.path.join(interm_dir, 'abtiters.2021.day0.pivoted.tsv.gz')
twentyone_data['abtiters'].drop('planned_day_relative_to_boost', axis=1).to_csv(abtiters_fn, index=False, sep='\t')

# save cytof 
cytof_fn = os.path.join(interm_dir, 'cytof.2021.day0.pivoted.tsv.gz')
twentyone_data['cytof'].drop('planned_day_relative_to_boost', axis=1).to_csv(cytof_fn, index=False, sep='\t')

# save olink
olink_fn = os.path.join(interm_dir, 'olink.2021.day0.pivoted.tsv.gz')
twentyone_data['olink'].drop('planned_day_relative_to_boost', axis=1).to_csv(olink_fn, index=False, sep='\t')

# save rnaseq
rnaseq_fn = os.path.join(interm_dir, 'rnaseq.2021.day0.pivoted.tsv.gz')
twentyone_data['rnaseq'].drop('planned_day_relative_to_boost', axis=1).to_csv(rnaseq_fn, index=False, sep='\t')

# save meta
meta_fn = os.path.join(interm_dir, 'meta.2021.pivoted.tsv.gz')
twentyone_data['master_meta'].to_csv(meta_fn, index=False, sep='\t')

In [14]:
# summary21 = []
# for assay, df in twentyone_data.items():
#     summary21.append([assay, df.subject_id.nunique()])

# summary20 = []
# for assay, df in twenty_data.items():
#     summary20.append([df.subject_id.nunique()])

# summary = pd.concat([pd.DataFrame(summary21), pd.DataFrame(summary20)], axis=1)
# summary.columns = ('table', 'nsamples21', 'nsamples20')
# summary = summary[['table', 'nsamples20', 'nsamples21']]

In [15]:
# fn = 'results/main/cmi_pb_datasets/2020LD_{}.csv'.format('olink_prot_exp')
# df = pd.read_table(fn, sep=',')

## Standardize RNA-seq

The RNA-seq dataset includes a lot more variables and processing so there are many steps we are including for standardization:

    1) remove zero variance genes,
    2) remove mitochondrial genes, 
    3) keep expressed genes (defined as those with TPM > 1) in more than a certain proportion of people (cut_filter)
    4) intersect genes across 2020 and 2021 and finally make harmonized 2020 and 2021 tables 

In [16]:
# load gene data to find mitochondrial genes
mito_genes = gencode.loc[(gencode.chr == 'chrM'), 'gene_nonversioned_id']
mito_genes = set(mito_genes.unique().tolist())

In [17]:
# value used across both RNA-seq datasets
cut_filter = 0.3

### RNA-seq for 2020

In [18]:
# find zero variance genes 
tmp = twenty_data['rnaseq'].loc[twenty_data['rnaseq'].planned_day_relative_to_boost == 0]
tmp = tmp.loc[:, tmp.columns.str.match('ENSG')]
tmpvars = tmp.var()
zero_var_genes = tmpvars[(tmpvars == 0)]
zero_var_genes = set(zero_var_genes.index.tolist())

In [19]:
# locate genes which are expressed
# defined as genes with TPM > 1 in more than 30% of samples 
expressed_genes = (tmp > 1).sum() > cut_filter * tmp.shape[0]
expressed_genes = set(expressed_genes.index[expressed_genes.values.tolist()])

In [20]:
# get the final list of genes 
keep_genes = expressed_genes.difference(mito_genes).difference(zero_var_genes)
keep_genes = keep_genes.intersection(gencode.gene_nonversioned_id.values.tolist())

In [21]:
# make the final new_rnaseq table
new_rnaseq1 = tmp.loc[:, ~tmp.columns.str.match('ENSG')]
new_rnaseq2 = tmp.loc[:, keep_genes]
twenty_data['rnaseq_std'] = pd.concat([new_rnaseq1, new_rnaseq2], axis=1)

### RNA-seq for 2021

In [22]:
# find zero variance genes 
tmp = twentyone_data['rnaseq'].loc[twentyone_data['rnaseq'].planned_day_relative_to_boost == 0]
tmp = tmp.loc[:, tmp.columns.str.match('ENSG')]
tmpvars = tmp.var()
zero_var_genes = tmpvars[(tmpvars == 0)]
zero_var_genes = set(zero_var_genes.index.tolist())

In [23]:
# locate genes which are expressed
# defined as genes with TPM > 1 in more than 30% of samples 
expressed_genes = (tmp > 1).sum() > cut_filter * tmp.shape[0]
expressed_genes = set(expressed_genes.index[expressed_genes.values.tolist()])

In [24]:
# get the final list of genes 
keep_genes = expressed_genes.difference(mito_genes).difference(zero_var_genes)
keep_genes = keep_genes.intersection(gencode.gene_nonversioned_id.values.tolist())

In [25]:
# make the final new_rnaseq table
new_rnaseq1 = tmp.loc[:, ~tmp.columns.str.match('ENSG')]
new_rnaseq2 = tmp.loc[:, keep_genes]
twentyone_data['rnaseq_std'] = pd.concat([new_rnaseq1, new_rnaseq2], axis=1)

### Harmonize the RNA-seq datasets 

In [26]:
# get the shared genes between 2020 and 2021
twenty_genes = set(twenty_data['rnaseq_std'].columns.str.extract('(ENSG[0-9]+)').dropna()[0])
twentyone_genes = set(twentyone_data['rnaseq_std'].columns.str.extract('(ENSG[0-9]+)').dropna()[0])
shared_genes = sorted(twenty_genes.intersection(twentyone_genes))

In [27]:
# getting harmonized for 2020 
new_rnaseq1 = twenty_data['rnaseq'].loc[:, ~twenty_data['rnaseq'].columns.str.match('ENSG')]
new_rnaseq2 = twenty_data['rnaseq'].loc[:, shared_genes]
harmonized_df = pd.concat([new_rnaseq1, new_rnaseq2], axis=1)
harmonized_df = harmonized_df.loc[harmonized_df.planned_day_relative_to_boost == 0]
harmonized_df.drop('planned_day_relative_to_boost', axis=1, inplace=True)
twenty_data['rnaseq_harmonized'] = harmonized_df

In [28]:
# getting harmonized for 2021
new_rnaseq1 = twentyone_data['rnaseq'].loc[:, ~twentyone_data['rnaseq'].columns.str.match('ENSG')]
new_rnaseq2 = twentyone_data['rnaseq'].loc[:, shared_genes]
harmonized_df = pd.concat([new_rnaseq1, new_rnaseq2], axis=1)
harmonized_df = harmonized_df.loc[harmonized_df.planned_day_relative_to_boost == 0]
harmonized_df.drop('planned_day_relative_to_boost', axis=1, inplace=True)
twentyone_data['rnaseq_harmonized'] = harmonized_df

In [29]:
# save harmonize rnaseq
harmony_dir = os.path.join(outdir, 'harmonized/')
os.makedirs(harmony_dir, exist_ok=True)

rnaseq_2020_fn = os.path.join(harmony_dir, 'rnaseq.2020.day0.pivoted.tsv.gz')
twenty_data['rnaseq_harmonized'].loc[:, ['subject_id'] + shared_genes].to_csv(rnaseq_2020_fn, index=False, sep='\t')

rnaseq_2021_fn = os.path.join(harmony_dir, 'rnaseq.2021.day0.pivoted.tsv.gz')
twentyone_data['rnaseq_harmonized'].loc[:, ['subject_id'] + shared_genes].to_csv(rnaseq_2021_fn, index=False, sep='\t')

## Standardize CyTOF

In [30]:
shared_cells = set(twenty_data['cytof'].columns).intersection(set(twentyone_data['cytof'].columns))
shared_cells = sorted(list(shared_cells))
shared_cells.remove('subject_id')
shared_cells.remove('planned_day_relative_to_boost')

In [31]:
day0_rows = twenty_data['cytof']['planned_day_relative_to_boost'] == 0
twenty_data['cytof_harmonized'] = twenty_data['cytof'].loc[day0_rows]
twenty_data['cytof_harmonized'] = twenty_data['cytof_harmonized'].loc[:, ['subject_id'] + shared_cells]

day0_rows = twentyone_data['cytof']['planned_day_relative_to_boost'] == 0
twentyone_data['cytof_harmonized'] = twentyone_data['cytof'].loc[day0_rows]
twentyone_data['cytof_harmonized'] = twentyone_data['cytof_harmonized'].loc[:, ['subject_id'] + shared_cells]

In [32]:
cytof_2020_fn = os.path.join(harmony_dir, 'cytof.2020.day0.pivoted.tsv.gz')
twenty_data['cytof_harmonized'].to_csv(cytof_2020_fn, index=False, sep='\t')

In [33]:
cytof_2021_fn = os.path.join(harmony_dir, 'cytof.2021.day0.pivoted.tsv.gz')
twentyone_data['cytof_harmonized'].to_csv(cytof_2021_fn, index=False, sep='\t')

## Standardize Ab Titers

In [34]:
# save abtiters 2020
old_abtiters_fn = os.path.abspath(os.path.join(interm_dir, 'abtiters.2020.day0.pivoted.tsv.gz'))
new_abtiters_fn = os.path.join(harmony_dir, 'abtiters.2020.day0.pivoted.tsv.gz')
if not os.path.islink(new_abtiters_fn):
    os.link(old_abtiters_fn, new_abtiters_fn)

# save abtiters 2021
old_abtiters_fn = os.path.abspath(os.path.join(interm_dir, 'abtiters.2021.day0.pivoted.tsv.gz'))
new_abtiters_fn = os.path.join(harmony_dir, 'abtiters.2021.day0.pivoted.tsv.gz')
if not os.path.islink(new_abtiters_fn):
    os.link(old_abtiters_fn, new_abtiters_fn)

## Standardize Olink

In [35]:
# save olink 2020
old_olink_fn = os.path.abspath(os.path.join(interm_dir, 'olink.2020.day0.pivoted.tsv.gz'))
new_olink_fn = os.path.join(harmony_dir, 'olink.2020.day0.pivoted.tsv.gz')
if not os.path.exists(new_olink_fn):
    os.link(old_olink_fn, new_olink_fn)

# save olink 2021
old_olink_fn = os.path.abspath(os.path.join(interm_dir, 'olink.2021.day0.pivoted.tsv.gz'))
new_olink_fn = os.path.join(harmony_dir, 'olink.2021.day0.pivoted.tsv.gz')
if not os.path.exists(new_olink_fn):
    os.link(old_olink_fn, new_olink_fn)

## Providing all task vectors for training

In [36]:
from IPython.display import display

In [37]:
# Loading the table describing each task 
tasks = pd.read_table('results/refs/tasks.tsv')

# Making a task vector using GENE IDS for RNA based tasks
i = 1
for idd, task in tasks.iterrows():
    
    # extracting the required columns
    cols = ['subject_id', task.fetchname]
    task_data = twenty_data[task.assay]
    task_data = task_data.loc[(task_data.planned_day_relative_to_boost == task.day), cols]
    task_data.columns = ['subject_id', task.fetchname + '_day' + str(task.day)]    
    
    # merging the data together
    if i == 1: 
        master_tasks = task_data
    else:
        master_tasks = master_tasks.merge(task_data, on='subject_id', how='outer')
    i += 1 

# saving the task matrix 
task_fn = os.path.join(harmony_dir, 'task_matrix.feature_names.tsv.gz')
master_tasks.to_csv(task_fn, sep='\t', index=False)

# Making a task vector using GENE NAMES for RNA based tasks
i = 1
for idd, task in tasks.iterrows():
    
    # extracting the required columns
    cols = ['subject_id', task.fetchname]
    task_data = twenty_data[task.assay]
    task_data = task_data.loc[(task_data.planned_day_relative_to_boost == task.day), cols]
    
    if task.assay == 'rnaseq':
        task_data.columns = ['subject_id', task.fullname + '_day' + str(task.day)]    
    else:
        task_data.columns = ['subject_id', task.fetchname + '_day' + str(task.day)]    
    
    # merging the data together
    if i == 1: 
        master_tasks = task_data
    else:
        master_tasks = master_tasks.merge(task_data, on='subject_id', how='outer')
    i += 1 

# saving the task matrix 
task_fn = os.path.join(harmony_dir, 'task_matrix.common_names.tsv.gz')
master_tasks.to_csv(task_fn, sep='\t', index=False)



In [38]:
master_tasks.head()

,subject_id,IgG-PT_day14,IgG-FHA_day14,IgG-PRN_day14,Monocytes_day1,ASCs (Plasmablasts)_day7,CD4Tcells_day3,CCL3_day3,IL-6_day3,NFKBIA_day7,XIST_day14
0,1,229.603692,0.064000,1039.814648,NaN,NaN,NaN,46.410,3.814,651.685,146.247
1,3,129.197956,640.894817,1030.816658,NaN,NaN,NaN,26.204,2.646,575.733,121.084
2,4,105.426574,162.807830,874.712337,7.211965,1.77,8.700555,13.353,3.770,550.870,0.132
3,5,97.743258,0.064000,646.092906,NaN,NaN,NaN,20.618,2.070,1098.404,0.066
4,6,162.500000,137.831579,58.431482,41.380502,3.50,41.706336,19.606,2.163,686.300,127.023


## Providing meta data into a basic format

In [39]:
twenty_basic_meta = twenty_data['master_meta'].drop_duplicates('subject_id')
drop_cols = ['specimen_id', 'actual_day_relative_to_boost', 'planned_day_relative_to_boost', 'visit', 'dataset']
twenty_basic_meta.drop(drop_cols, axis=1, inplace=True)

twenty_basic_fn = os.path.join(harmony_dir, 'clinical_metadata.2020.tsv.gz')
twenty_basic_meta.to_csv(twenty_basic_fn, sep='\t', index=False)

/mnt/BioHome/jreyna/.local/lib/python3.6/site-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [40]:
twentyone_basic_meta = twentyone_data['master_meta'].drop_duplicates('subject_id')
twentyone_basic_meta.drop(drop_cols, axis=1, inplace=True)

twentyone_basic_fn = os.path.join(harmony_dir, 'clinical_metadata.2021.tsv.gz')
twentyone_basic_meta.to_csv(twentyone_basic_fn, sep='\t', index=False)